In [87]:
import numpy as np

def mae(pred, gold, index_list=None):
    if index_list is not None:
        pred = pred[:,:,index_list]
        gold = gold[:,:,index_list]
    return np.abs(pred-gold).mean()

def rmse(pred, gold, index_list=None):
    if index_list is not None:
        pred = pred[:,:,index_list]
        gold = gold[:,:,index_list]
    return np.sqrt(((pred-gold)*(pred-gold)).mean())

def smape(pred, gold, index_list=None):
    if index_list is not None:
        pred = pred[:,:,index_list]
        gold = gold[:,:,index_list]
    return 100 * np.mean(np.abs(pred - gold) / (np.abs(pred) + np.abs(gold)))

def rrmse(pred, gold, index_list=None):
    if index_list is not None:
        pred = pred[:,:,index_list]
        gold = gold[:,:,index_list]
    rrmse_loss = np.sqrt(((pred-gold)*(pred-gold)).mean() / np.square(pred).mean())
    return rrmse_loss * 100

def fairness(pred, gold, skill_num = 2335):
    attribute_num = pred.shape[-1] // skill_num
    pred = pred.reshape(-1, attribute_num)
    gold = gold.reshape(-1, attribute_num)
    mae = np.mean(np.abs(pred - gold), axis=0)
    variance = np.var(mae)
    return variance

def compute_metrics(pred, gold, index_list=None, tag=''):
    metrics = {}
    metrics[f'{tag}mae'] = mae(pred, gold, index_list)
    metrics[f'{tag}rmse'] = rmse(pred, gold, index_list)
    metrics[f'{tag}smape'] = smape(pred, gold, index_list)
    metrics[f'{tag}rrmse'] = rrmse(pred, gold, index_list)
    if index_list is None:
        metrics[f'{tag}variance'] = fairness(pred, gold)
    return metrics

In [88]:
import json
import pandas as pd
root_path = '/data/chenxi/research/conference/NeurIPS24/code/benchmark/multivariate_time_series/results'
index_path = '/data/chenxi/research/conference/NeurIPS24/code/dataset'
itr = 0
ans = {}
for dataset in ['r1', 'r2', 'region']:
    ans[dataset] = {}
    structural_breaks_index = json.load(open(f"{index_path}/structural_breaks_index/{dataset}.json", 'r'))
    low_frequency_index = json.load(open(f"{index_path}/low_frequency_index/{dataset}.json", 'r'))
    for model_name in ['PatchTST', 'FiLM', 'Koopa', 'DLinear', 'SegRNN']:
        for task_mode in ['', '_fair', '_break_tukey-hanning']:
            pred = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/pred.npy')
            gold = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/true.npy')
            ans[dataset][f"{model_name}{task_mode}"] = compute_metrics(pred, gold)
            ans[dataset][f"{model_name}{task_mode}"].update(compute_metrics(pred, gold, low_frequency_index, 'low-'))
            ans[dataset][f"{model_name}{task_mode}"].update(compute_metrics(pred, gold, structural_breaks_index, 'break-'))
    pd.DataFrame(ans[dataset]).T.to_csv(f"results/whole/{dataset}.csv")

In [89]:
import json
import pandas as pd
root_path = '/data/chenxi/research/conference/NeurIPS24/code/benchmark/multivariate_time_series/results'
index_path = '/data/chenxi/research/conference/NeurIPS24/code/dataset'
itr = 0
ans = {}
df_list = []
for dataset in ['region', 'r1']:
    ans[dataset] = {}
    for model_name in ['PatchTST', 'Koopa', 'SegRNN']:
        for task_mode in ['', '_fair']:
            pred = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/pred.npy')
            gold = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/true.npy')
            ans[dataset][f"{model_name}{task_mode}"] = compute_metrics(pred, gold)
    df = pd.DataFrame(ans[dataset]).T.reset_index()
    df['Granularity'] = dataset
    df.columns = ['Metric', 'MAE', 'RMSE', 'SMAPE', 'RRMSE', 'VAR', 'Granularity']
    df = df.set_index(['Granularity'])
    df_list.append(df)
df_list = pd.concat(df_list, axis=0) 
markdown_table = df_list.to_markdown(index=True).replace('r1', 'L1-Occupation').replace('r2', 'L2-Occupation').replace('region', 'Region')
with open(f"results/fair/1.txt", 'w') as w:
    w.write(markdown_table)

In [91]:
import json
import pandas as pd
root_path = '/data/chenxi/research/conference/NeurIPS24/code/benchmark/multivariate_time_series/results'
index_path = '/data/chenxi/research/conference/NeurIPS24/code/dataset'
itr = 0
ans = {}
df_list = []
for dataset in ['region','r1']:
    ans[dataset] = {}
    structural_breaks_index = json.load(open(f"{index_path}/structural_breaks_index/{dataset}.json", 'r'))
    for model_name in ['PatchTST', 'Koopa', 'SegRNN']:
        for task_mode in ['',  '_break_parzen']:
            pred = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/pred.npy')
            gold = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/true.npy')
            ans[dataset][f"{model_name}{task_mode}"] = compute_metrics(pred, gold, structural_breaks_index)
    df = pd.DataFrame(ans[dataset]).T.reset_index()
    df.columns = ['Metric', 'MAE', 'RMSE', 'SMAPE', 'RRMSE']
    df['Granularity'] = dataset
    df = df.set_index(['Granularity'])
    df_list.append(df)
df_list = pd.concat(df_list, axis=0) 
markdown_table = df_list.to_markdown(index=True).replace('r1', 'L1-Occupation').replace('r2', 'L2-Occupation').replace('region', 'Region').replace('_break_parzen', '_break')
with open(f"results/break/1.txt", 'w') as w:
    w.write(markdown_table)

In [93]:
import json
import pandas as pd
root_path = '/data/chenxi/research/conference/NeurIPS24/code/benchmark/multivariate_time_series/results'
index_path = '/data/chenxi/research/conference/NeurIPS24/code/dataset'
itr = 0
ans = {}
df_list = []
for dataset in ['region', 'r1']:
    ans[dataset] = {}
    for model_name in ['PatchTST', 'Koopa', 'SegRNN']:
        for task_mode in ['', '_fair', '_low']:
            pred = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/pred.npy')
            gold = np.load(f'{root_path}/long_term_forecast{task_mode}_0_{model_name}_job_demand_{dataset}_ftM_sl6_ll1_pl3_dm512_nh8_el2_dl2_df2048_expand2_dc4_fc1_eblearned_dtTrue_test_{itr}/true.npy')
            ans[dataset][f"{model_name}{task_mode}"] = compute_metrics(pred, gold)
    df = pd.DataFrame(ans[dataset]).T.reset_index()
    df['Granularity'] = dataset
    df.columns = ['Metric', 'MAE', 'RMSE', 'SMAPE', 'RRMSE', 'VAR', 'Granularity']
    df = df.set_index(['Granularity'])
    df_list.append(df)
df_list = pd.concat(df_list, axis=0) 
markdown_table = df_list.to_markdown(index=True).replace('r1', 'L1-Occupation').replace('r2', 'L2-Occupation').replace('region', 'Region')
with open(f"results/low/1.txt", 'w') as w:
    w.write(markdown_table)